In [1]:
import pandas as pd 
import numpy as np
import csv
import baseline_functions as base

# restore saved variables
%store -r summary_violent6_ky_model

no stored variable summary_violent6_ky_model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_violence_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_violence_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [40,60]
depth = [6,7]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators,
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [8]
n_estimators = [200]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_violent6_ky_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_violent6_ky_model

Stored 'summary_violent6_ky_model' (dict)


In [4]:
results = []

for model_name, model_summary in summary_violent6_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['FL_score']])
results

[['Logistic',
  0.8655218636130393,
  0.0024670420738700516,
  {'C': 0.01},
  0.6639226247946204],
 ['Lasso',
  0.865359420853768,
  0.0020315683474134705,
  {'C': 0.01},
  0.6599827226993241],
 ['LinearSVM',
  0.8659997953952229,
  0.0023374425696104373,
  {'C': 0.1},
  0.6358758066974948],
 ['RF',
  0.8707031365616469,
  0.016788984992058165,
  {'max_depth': 7, 'n_estimators': 60},
  0.6613699883124142],
 ['XGBoost',
  0.865536906421032,
  0.020117812819731573,
  {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 200},
  0.6295543472737437]]

In [5]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\Six Month\\"
results = [["Violent",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Six Month KY_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)